<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [ ]:
!pip install category_encoders

In [53]:
# imports
import pandas as pd
import category_encoders as ce

In [83]:
# loading the data
FILEPATH = 'WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'

df = pd.read_csv(FILEPATH)

In [85]:
df['tenure']

0        1
1       34
2        2
3       45
4        2
        ..
7038    24
7039    72
7040    11
7041     4
7042    66
Name: tenure, Length: 7043, dtype: int64

In [86]:
# cleaning data

# transforming binary columns
columns = df.columns
map2binary = {'Female': 0,
              'Male': 1,
              'No': 0,
              'Yes': 1,
              0:0,
              1:1,
              'No phone service': 0,
              'No internet service': 0}

for each in columns:
    if df[each].value_counts().shape[0] == 2:
        df[each] = df[each].map(map2binary)

df['MultipleLines'] = df['MultipleLines'].map(map2binary)
df['OnlineSecurity'] = df['OnlineSecurity'].map(map2binary)
df['OnlineBackup'] = df['OnlineBackup'].map(map2binary)
df['DeviceProtection'] = df['DeviceProtection'].map(map2binary)
df['TechSupport'] = df['TechSupport'].map(map2binary)
df['StreamingTV'] = df['StreamingTV'].map(map2binary)
df['StreamingMovies'] = df['StreamingMovies'].map(map2binary)

# one hot encoding
columns_to_onehot = ['InternetService', 'Contract', 'PaymentMethod']
encoder = ce.OneHotEncoder(use_cat_names=True)
encoded_df = encoder.fit_transform(df[columns_to_onehot])

,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic)
0,1,0,0,1,0,0,1,0,0,0
1,1,0,0,0,1,0,0,1,0,0
2,1,0,0,1,0,0,0,1,0,0
3,1,0,0,0,1,0,0,0,1,0
4,0,1,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7038,1,0,0,0,1,0,0,1,0,0
7039,0,1,0,0,1,0,0,0,0,1
7040,1,0,0,1,0,0,1,0,0,0
7041,0,1,0,1,0,0,0,1,0,0


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?